<a href="https://colab.research.google.com/github/keppy/KeppyLab/blob/master/Copy_of_Byte_Pair_Encoding_tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Byte-Pair Encoding tokenization

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.4 MB/s eta 0:00:00


In [ ]:
corpus = [
    "μῆνιν ἄειδε θεὰ Πηληϊάδεω Ἀχιλῆος",
    "οὐλομένην, ἣ μυρί᾽ Ἀχαιοῖς ἄλγε᾽ ἔθηκε,",
    "πολλὰς δ᾽ ἰφθίμους ψυχὰς Ἄϊδι προΐαψεν",
    "ἡρώων, αὐτοὺς δὲ ἑλώρια τεῦχε κύνεσσιν",
    "βάλλ᾽: αἰεὶ δὲ πυραὶ νεκύων καίοντο θαμειαί.",
]
print(corpus)

['μῆνιν ἄειδε θεὰ Πηληϊάδεω Ἀχιλῆος', 'οὐλομένην, ἣ μυρί᾽ Ἀχαιοῖς ἄλγε᾽ ἔθηκε,', 'πολλὰς δ᾽ ἰφθίμους ψυχὰς Ἄϊδι προΐαψεν', 'ἡρώων, αὐτοὺς δὲ ἑλώρια τεῦχε κύνεσσιν', 'βάλλ᾽: αἰεὶ δὲ πυραὶ νεκύων καίοντο θαμειαί.']


In [ ]:
# This won't work, it doesn't keep the unicode Greek characters
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
a = [('a', (0, 5))]
print(a)
import re
re.findall(r"\w+|[^\w\s]", corpus[0])


[('a', (0, 5))]


['μῆνιν', 'ἄειδε', 'θεὰ', 'Πηληϊάδεω', 'Ἀχιλῆος']

For pre-tokens that are words $t$ we observe that the number of spaces in a batch, which we call $n$, can be applied over the inverse of the sum:


\begin{equation}
g = \sum_{i=0}^{len(n)} -t_i n_i
\end{equation}

This is the Greek Trick

In this way we can blow away the spaces and then bring them back to life by observing that the first word in a text batch is the least likely to have a space prepended, while the last word is most likely to have a space prepended.

We prove that this probability holds because if we look at The Illiad no lines start with a space, and every word at the end of a line and its preceeding words have white space prepended. On additional texts I would suggest testing a scheme that pre-processes the text into blocks that are least likey to have spaces prepended to the head item. Uppercase and other gramar signal inputs could be used to make the pre-processing step more robust.

In [ ]:
from pickle import FALSE
from collections import defaultdict
import re
import string

greek_puctuation = [':', '·', '…', '\'', '«', '»', '-', ';', '!', ',', '(', ')', '᾽']
word_freqs = defaultdict(int)
words_with_offsets = []
last = 0
first = 0
n = 0
g = []
has_space = FALSE
space_delim = 'Ġ'

for batch in corpus:
    t = re.findall(r"\w+|[^\w\s]", batch)
    n = batch.count(' ')

    for idx, ti in enumerate(reversed(t)):
        if not ti in greek_puctuation and idx < len(t)-1:
            n-1
            g.append(space_delim+ti)
        else:
            g.append(ti)
    g.reverse()
    print(g)

    for idx, word in enumerate(g):
        if last == 0:
          last = len(word)
        else:
          last = len(word)+last
        words_with_offsets.append((word, (first, last)))
        first = last

    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

    words_with_offsets = []
    g  = []

print(n)
print(word_freqs)

['μῆνιν', 'Ġἄειδε', 'Ġθεὰ', 'ĠΠηληϊάδεω', 'ĠἈχιλῆος']
['οὐλομένην', ',', 'Ġἣ', 'Ġμυρί', '᾽', 'ĠἈχαιοῖς', 'Ġἄλγε', '᾽', 'Ġἔθηκε', ',']
['πολλὰς', 'Ġδ', '᾽', 'Ġἰφθίμους', 'Ġψυχὰς', 'ĠἌϊδι', 'Ġπροΐαψεν']
['ἡρώων', ',', 'Ġαὐτοὺς', 'Ġδὲ', 'Ġἑλώρια', 'Ġτεῦχε', 'Ġκύνεσσιν']
['βάλλ', '᾽', ':', 'Ġαἰεὶ', 'Ġδὲ', 'Ġπυραὶ', 'Ġνεκύων', 'Ġκαίοντο', 'Ġθαμειαί', 'Ġ.']
6
defaultdict(<class 'int'>, {'μῆνιν': 1, 'Ġἄειδε': 1, 'Ġθεὰ': 1, 'ĠΠηληϊάδεω': 1, 'ĠἈχιλῆος': 1, 'οὐλομένην': 1, ',': 3, 'Ġἣ': 1, 'Ġμυρί': 1, '᾽': 4, 'ĠἈχαιοῖς': 1, 'Ġἄλγε': 1, 'Ġἔθηκε': 1, 'πολλὰς': 1, 'Ġδ': 1, 'Ġἰφθίμους': 1, 'Ġψυχὰς': 1, 'ĠἌϊδι': 1, 'Ġπροΐαψεν': 1, 'ἡρώων': 1, 'Ġαὐτοὺς': 1, 'Ġδὲ': 2, 'Ġἑλώρια': 1, 'Ġτεῦχε': 1, 'Ġκύνεσσιν': 1, 'βάλλ': 1, ':': 1, 'Ġαἰεὶ': 1, 'Ġπυραὶ': 1, 'Ġνεκύων': 1, 'Ġκαίοντο': 1, 'Ġθαμειαί': 1, 'Ġ.': 1})


In [ ]:
alphabet = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

print(alphabet)

[',', '.', ':', 'ΐ', 'Π', 'ά', 'έ', 'ί', 'α', 'β', 'γ', 'δ', 'ε', 'η', 'θ', 'ι', 'κ', 'λ', 'μ', 'ν', 'ο', 'π', 'ρ', 'ς', 'σ', 'τ', 'υ', 'φ', 'χ', 'ψ', 'ω', 'ϊ', 'ύ', 'ώ', 'ἄ', 'Ἀ', 'Ἄ', 'ἑ', 'ἔ', 'ἡ', 'ἣ', 'ἰ', 'ὐ', 'ὰ', 'ὲ', 'ὶ', 'ὺ', '᾽', 'ῆ', 'ῖ', 'ῦ']


In [ ]:
vocab = ["<|endoftext|>"] + alphabet.copy()

In [ ]:
splits = {word: [c for c in word] for word in word_freqs.keys()}

In [ ]:
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

In [ ]:
pair_freqs = compute_pair_freqs(splits)

for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

('T', 'h'): 3
('h', 'i'): 3
('i', 's'): 5
('Ġ', 'i'): 2
('Ġ', 't'): 7
('t', 'h'): 3

In [ ]:
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('Ġ', 't') 7

In [ ]:
merges = {("Ġ", "t"): "Ġt"}
vocab.append("Ġt")

In [ ]:
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

In [ ]:
splits = merge_pair("Ġ", "t", splits)
print(splits["Ġtrained"])

['Ġt', 'r', 'a', 'i', 'n', 'e', 'd']

In [ ]:
vocab_size = 50

while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [ ]:
print(merges)

{('Ġ', 't'): 'Ġt', ('i', 's'): 'is', ('e', 'r'): 'er', ('Ġ', 'a'): 'Ġa', ('Ġt', 'o'): 'Ġto', ('e', 'n'): 'en',
 ('T', 'h'): 'Th', ('Th', 'is'): 'This', ('o', 'u'): 'ou', ('s', 'e'): 'se', ('Ġto', 'k'): 'Ġtok',
 ('Ġtok', 'en'): 'Ġtoken', ('n', 'd'): 'nd', ('Ġ', 'is'): 'Ġis', ('Ġt', 'h'): 'Ġth', ('Ġth', 'e'): 'Ġthe',
 ('i', 'n'): 'in', ('Ġa', 'b'): 'Ġab', ('Ġtoken', 'i'): 'Ġtokeni'}

In [ ]:
print(vocab)

['<|endoftext|>', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o',
 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'Ġ', 'Ġt', 'is', 'er', 'Ġa', 'Ġto', 'en', 'Th', 'This', 'ou', 'se',
 'Ġtok', 'Ġtoken', 'nd', 'Ġis', 'Ġth', 'Ġthe', 'in', 'Ġab', 'Ġtokeni']

In [ ]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])

In [ ]:
tokenize("This is not a token.")

['This', 'Ġis', 'Ġ', 'n', 'o', 't', 'Ġa', 'Ġtoken', '.']